In [1]:
# default_exp data.finance

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from libs import is_lib_exists

In [4]:
required_libs = [ ("numpy", "numpy"),
                  ("pandas", "pandas"),
                  ("seaborn", "seaborn"),
                  ("matplotlib", "matplotlib"),
                  ("investpy", "investpy"),
                  ("dateutil", "python-dateutil"),
                ]

In [5]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
investpy exists
python-dateutil exists


In [6]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import investpy
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
import calendar

In [7]:
%matplotlib inline

In [8]:
# % - line, %% - cell
# %run, %%timeit, %%writefile, %pycat
# %who, %store
# %html 
# %env, %pinfo
# %lsmagic
from IPython.display import display, Image

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [9]:
#hide
from nbdev.showdoc import *

In [10]:
# Dataset path
DATASET_PATH = Path("../../data")  

# Country to analyze
COUNTRY = "malaysia"

# Dataset
DATASET = DATASET_PATH/f"ftse_klse_investing.csv"

## FTSE KLSE

In [11]:
# Retrieve stock components
dataset = investpy.get_stocks(country=COUNTRY)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    929 non-null    object
 1   name       929 non-null    object
 2   full_name  929 non-null    object
 3   isin       929 non-null    object
 4   currency   929 non-null    object
 5   symbol     929 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [12]:
# Save the dataset
dataset.to_csv(DATASET, index=False)

#dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"')

In [32]:
display(dataset.head(3))

,country,name,full_name,isin,currency,symbol
0,malaysia,Media Chinese Int,Media Chinese International Ltd,BMG5959D1048,MYR,MDCH
1,malaysia,AMMB,AMMB Holdings Bhd,MYL1015OO006,MYR,AMMB
2,malaysia,CIMB Group,CIMB Group Holdings Bhd,MYL1023OO000,MYR,CIMB


In [46]:
# Today's date
TODAY = date.today()

# Past 3 years
PAST_3_YEARS = [ TODAY.year - y for y in range(1,4) ]

# Expected YIELD %
EXPECTED_YIELD_PERCENTAGE = 6

In [47]:
def expected_yield(name, symbol):
    df_dividends = investpy.get_stock_dividends(stock=symbol, country=COUNTRY)
    df_dividends['Year'] = pd.DatetimeIndex(df_dividends['Date']).year
    display(df_dividends)
    # Aggregate the dividends by years
    df_dividends_summary = df_dividends.groupby(by=['Year'], sort=False, as_index=False)['Dividend'].sum()
    display(df_dividends_summary[df_dividends_summary['Year'].isin(PAST_3_YEARS)])
    
for _, row in dataset.iterrows():
    name = row['name']
    symbol = row['symbol']
    expected_yield(name, symbol)
    break

,Date,Dividend,Type,Payment Date,Yield,Year
0,2019-12-09,0.0109,trailing_twelve_months,2019-12-30,"5,59%",2019
1,2019-06-19,0.0117,trailing_twelve_months,2019-07-12,"7,31%",2019
2,2018-12-12,0.0151,trailing_twelve_months,2018-12-28,"7,95%",2018
3,2018-06-18,0.0171,trailing_twelve_months,2018-07-13,"8,55%",2018
4,2017-12-11,0.0250,trailing_twelve_months,2017-12-29,"8,47%",2017
5,2017-06-16,0.0307,trailing_twelve_months,2017-07-10,"7,31%",2017
6,2016-12-13,0.0428,trailing_twelve_months,2016-12-30,"7,25%",2016
7,2016-06-20,0.0451,trailing_twelve_months,2016-07-13,"6,35%",2016
8,2015-12-08,0.0451,trailing_twelve_months,2015-12-23,"6,18%",2015
9,2015-07-08,0.0339,trailing_twelve_months,2015-07-31,-,2015


,Year,Dividend
0,2019,0.0226
1,2018,0.0322
2,2017,0.0557


In [15]:
df_dividends = investpy.get_stock_dividends(stock='MDCH', country=COUNTRY)
df_dividends['Year'] = pd.DatetimeIndex(df_dividends['Date']).year
display(df_dividends)

,Date,Dividend,Type,Payment Date,Yield,Year
0,2019-12-09,0.0109,trailing_twelve_months,2019-12-30,"5,59%",2019
1,2019-06-19,0.0117,trailing_twelve_months,2019-07-12,"7,31%",2019
2,2018-12-12,0.0151,trailing_twelve_months,2018-12-28,"7,95%",2018
3,2018-06-18,0.0171,trailing_twelve_months,2018-07-13,"8,55%",2018
4,2017-12-11,0.0250,trailing_twelve_months,2017-12-29,"8,47%",2017
5,2017-06-16,0.0307,trailing_twelve_months,2017-07-10,"7,31%",2017
6,2016-12-13,0.0428,trailing_twelve_months,2016-12-30,"7,25%",2016
7,2016-06-20,0.0451,trailing_twelve_months,2016-07-13,"6,35%",2016
8,2015-12-08,0.0451,trailing_twelve_months,2015-12-23,"6,18%",2015
9,2015-07-08,0.0339,trailing_twelve_months,2015-07-31,-,2015


In [16]:
# Aggregate the dividends by years
df_dividends_summary = df_dividends.groupby(by=['Year'], sort=False, as_index=False)['Dividend'].sum()
display(df_dividends_summary[df_dividends_summary['Year'].isin(PAST_3_YEARS)])

,Year,Dividend
0,2019,0.0226
1,2018,0.0322
2,2017,0.0557


In [17]:
df_recent_data = investpy.get_stock_recent_data(stock='MDCH', country=COUNTRY).reset_index()
df_recent_data

,Date,Open,High,Low,Close,Volume,Currency
0,2020-05-27,0.185,0.190,0.180,0.185,457700,MYR
1,2020-05-28,0.185,0.210,0.180,0.200,18857800,MYR
2,2020-05-29,0.200,0.215,0.185,0.195,14322300,MYR
3,2020-06-01,0.190,0.200,0.190,0.190,1513300,MYR
4,2020-06-02,0.190,0.195,0.190,0.195,1129600,MYR
5,2020-06-03,0.200,0.205,0.195,0.200,12295800,MYR
6,2020-06-04,0.200,0.205,0.195,0.195,1379000,MYR
7,2020-06-05,0.195,0.200,0.190,0.190,9925500,MYR
8,2020-06-09,0.195,0.200,0.190,0.200,1702700,MYR
9,2020-06-10,0.200,0.205,0.195,0.200,3564700,MYR


In [18]:
last_close_price = df_recent_data[df_recent_data['Date'].eq(df_recent_data['Date'].max())]['Close'].values[0]
last_close_price

0.195

In [19]:
df_dividends_summary['Last Close Price'] = last_close_price
df_dividends_summary['Yield'] = df_dividends_summary['Dividend'] / last_close_price * 100
df_dividends_summary

,Year,Dividend,Last Close Price,Yield
0,2019,0.0226,0.195,11.589744
1,2018,0.0322,0.195,16.512821
2,2017,0.0557,0.195,28.564103
3,2016,0.0879,0.195,45.076923
4,2015,0.0790,0.195,40.512821


In [31]:
df_desired_returns = df_dividends_summary[
                    (df_dividends_summary['Year'].isin(PAST_3_YEARS)) & 
                    (df_dividends_summary['Yield'] >= EXPECTED_YIELD_PERCENTAGE)]
df_desired_returns

,Year,Dividend,Last Close Price,Yield
0,2019,0.0226,0.195,11.589744
1,2018,0.0322,0.195,16.512821
2,2017,0.0557,0.195,28.564103
